## Variación espacio-temporal precipitación total

**PROYECTO:** SISTEMA PARA EL SEGUIMIENTO DE ECOSISTEMAS VENEZOLANOS \
**AUTOR:** Javier Martinez

Directorio

In [1]:
import os

print('> Directorio actual: ', os.getcwd())  
os.chdir('../')
print('> Directorio actual: ', os.getcwd()) 

> Directorio actual:  /media/javier/Compartida/doctorado/ssev-analytics/cerro_saroche
> Directorio actual:  /media/javier/Compartida/doctorado/ssev-analytics


In [2]:
from utils.MONGO import CONEXION
from utils.UTILS import *
from datetime import datetime
import pandas as pd

from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [3]:
import warnings
warnings.filterwarnings('ignore')

Datos de precipitación 

In [4]:
park = 'cerro_saroche'

In [5]:
pd_precipitacion = pd.read_pickle(f'./{park}/data/narx_precipitacion.pkl')
pd_precipitacion['precipitacion_narx'] = pd_precipitacion['prediction_precipitacion_mm']
pd_precipitacion.head()

,park,periodo,year,month,id_point,latitud,longitud,type,precipitacion_mm,prediction_precipitacion_mm,elevacion_media,precipitacion_narx
0,cerro_saroche,1995-01-01,1995,1,1,10.31,-69.83,training,0.072351,0.176529,508.541046,0.176529
1,cerro_saroche,1995-02-01,1995,2,1,10.31,-69.83,training,0.340843,0.335624,508.541046,0.335624
2,cerro_saroche,1995-03-01,1995,3,1,10.31,-69.83,training,2.290730,1.018381,508.541046,1.018381
3,cerro_saroche,1995-04-01,1995,4,1,10.31,-69.83,training,1.064486,1.298513,508.541046,1.298513
4,cerro_saroche,1995-05-01,1995,5,1,10.31,-69.83,training,1.114330,1.298577,508.541046,1.298577


In [6]:
pd_historical = pd_precipitacion[pd_precipitacion.type.isin(['training', #'test',
                                                            'test',
                                                            'prediction'])]
pd_historical.head()

,park,periodo,year,month,id_point,latitud,longitud,type,precipitacion_mm,prediction_precipitacion_mm,elevacion_media,precipitacion_narx
0,cerro_saroche,1995-01-01,1995,1,1,10.31,-69.83,training,0.072351,0.176529,508.541046,0.176529
1,cerro_saroche,1995-02-01,1995,2,1,10.31,-69.83,training,0.340843,0.335624,508.541046,0.335624
2,cerro_saroche,1995-03-01,1995,3,1,10.31,-69.83,training,2.290730,1.018381,508.541046,1.018381
3,cerro_saroche,1995-04-01,1995,4,1,10.31,-69.83,training,1.064486,1.298513,508.541046,1.298513
4,cerro_saroche,1995-05-01,1995,5,1,10.31,-69.83,training,1.114330,1.298577,508.541046,1.298577


Preparando los datos

In [7]:
variables = ['year',	'month',	'latitud',	'longitud', 'elevacion_media']

In [8]:
pd_historical[variables+['precipitacion_narx']].corr()

,year,month,latitud,longitud,elevacion_media,precipitacion_narx
year,1.000000e+00,-2.550058e-02,8.237437e-14,-3.993847e-15,4.556907e-14,-0.037488
month,-2.550058e-02,1.000000e+00,4.077476e-16,-8.901068e-17,2.768332e-16,0.286153
latitud,8.237437e-14,4.077476e-16,1.000000e+00,-4.612536e-15,-9.997471e-02,0.021666
longitud,-3.993847e-15,-8.901068e-17,-4.612536e-15,1.000000e+00,7.713106e-01,0.155172
elevacion_media,4.556907e-14,2.768332e-16,-9.997471e-02,7.713106e-01,1.000000e+00,0.098641
precipitacion_narx,-3.748813e-02,2.861531e-01,2.166555e-02,1.551720e-01,9.864085e-02,1.000000


In [9]:
trans_variable = MinMaxScaler()

variables = ['year',	'month',	'latitud',	'longitud', 'elevacion_media']

trans_variable.fit(pd_historical[variables])

data_model = pd_historical[variables].copy()

data_model[variables] = trans_variable.transform(pd_historical[variables]).astype(float)

data_model.head()

,year,month,latitud,longitud,elevacion_media
0,0.0,0.000000,1.0,0.0,0.0
1,0.0,0.090909,1.0,0.0,0.0
2,0.0,0.181818,1.0,0.0,0.0
3,0.0,0.272727,1.0,0.0,0.0
4,0.0,0.363636,1.0,0.0,0.0


In [10]:
# Transformacion
transformacion = LogMinimax.create( pd_historical['precipitacion_narx'].to_numpy() )

data_model['precipitacion_narx'] = transformacion.transformacion()

data_model.head()

,year,month,latitud,longitud,elevacion_media,precipitacion_narx
0,0.0,0.000000,1.0,0.0,0.0,0.390347
1,0.0,0.090909,1.0,0.0,0.0,0.485247
2,0.0,0.181818,1.0,0.0,0.0,0.649193
3,0.0,0.272727,1.0,0.0,0.0,0.685085
4,0.0,0.363636,1.0,0.0,0.0,0.685093


## Red ANN

Data entrenamiento y validación

https://www.section.io/engineering-education/build-ann-with-keras/

In [11]:
test_size = 0.2
random_state = 0
X_train, X_test, y_train, y_test = train_test_split(data_model[variables].to_numpy(), 
                                                    data_model['precipitacion_narx'].to_numpy(), 
                                                    test_size = test_size, 
                                                    random_state = random_state,
                                                    stratify=data_model[['latitud',	'longitud']])

Generando la red

In [12]:
activation = ['sigmoid','sigmoid']

total = int(2*X_train.shape[1]/3)

n_neuronas = [total-1, 1]
epochs = 10
patience = 3

In [13]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [14]:
# Metrícas
mae = keras.metrics.MeanAbsoluteError()
rmse = keras.metrics.RootMeanSquaredError()

# ANN
model = keras.models.Sequential()

model.add(keras.layers.Dense(units=n_neuronas[0], activation=activation[0], input_shape=X_train[0].shape))
model.add(keras.layers.Dropout(0.1))

if len(n_neuronas) > 1:
    for index in range(1,len(n_neuronas)):
    
        model.add(keras.layers.Dense(units=n_neuronas[index], activation=activation[index]))

# Compilando modelo
model.compile(loss='mean_squared_error', optimizer='adam', metrics=[mae,rmse]) 

# Parada temprana
callback = keras.callbacks.EarlyStopping(
                                            monitor="loss",
                                            min_delta=0,
                                            patience=patience,
                                            verbose=0,
                                            mode="min",
                                            baseline=None,
                                            restore_best_weights=False,
                                        )

Entrenamiento

In [15]:
history = model.fit(x=X_train,
                    y=y_train,
                    epochs=epochs,
                    batch_size=1,
                    verbose=0,
                    workers=2,
                    callbacks=[callback])

print(f'Total epocas:{len(history.epoch)}')

Total epocas:10


Validación

In [16]:
# make predictions
trainPredict = model.predict(X_train, verbose=0).reshape(-1)
testPredict = model.predict(X_test, verbose=0).reshape(-1)

In [17]:
def time_dormat(year = 2003, month = 1):
    """
    Funcion para calcular el periodo
    """

    if month < 10.:
        month = '0'+str(int(month))
    else:
        month = str(int(month))

    return str(int(year))+'-'+month+'-'+'01'

In [18]:
# Training
trainind_pd = pd.DataFrame(y_train,
                            index = list(range(y_train.shape[0])) ,
                            columns=['precipitacion_narx']
                            )

trainind_pd[variables] = X_train
trainind_pd['prediction_ann'] = trainPredict

trainind_pd[variables] = trans_variable.inverse_transform(trainind_pd[variables])
trainind_pd['precipitacion_narx'] = trainind_pd['precipitacion_narx'].apply(lambda x: transformacion.inversa(x) if np.isnan(x)==False else np.nan )
trainind_pd['prediction_ann'] = trainind_pd['prediction_ann'].apply(lambda x: transformacion.inversa(x) if np.isnan(x)==False else np.nan )

trainind_pd[['year','month']] = trainind_pd[['year','month']].round(0)

trainind_pd['periodo'] = trainind_pd[['year','month']].apply(lambda x: time_dormat(year = int(x.year), month = int(x.month) ),1)
trainind_pd['periodo'] = pd.to_datetime(trainind_pd['periodo'] )

trainind_pd['type'] = 'training'

# Validacion entrenamiento
training_metrics = metrics(observado=trainind_pd['precipitacion_narx'],
                           prediccion=trainind_pd['prediction_ann'])

In [19]:
# Test
test_pd = pd.DataFrame(y_test,
                        index = list(range(y_test.shape[0])) ,
                        columns=['precipitacion_narx']
                        )

test_pd[variables] = X_test
test_pd['prediction_ann'] = testPredict

test_pd[variables] = trans_variable.inverse_transform(test_pd[variables])
test_pd['precipitacion_narx'] = test_pd['precipitacion_narx'].apply(lambda x: transformacion.inversa(x) if np.isnan(x)==False else np.nan )
test_pd['prediction_ann'] = test_pd['prediction_ann'].apply(lambda x: transformacion.inversa(x) if np.isnan(x)==False else np.nan )

test_pd[['year','month']] = test_pd[['year','month']].round(0)
test_pd['periodo'] = test_pd[['year','month']].apply(lambda x: time_dormat(year = x.year, month = x.month),1)
test_pd['periodo'] = pd.to_datetime(test_pd['periodo'] )

test_pd['type'] = 'test'

# Validacion entrenamiento
test_metrics = metrics(observado=test_pd['precipitacion_narx'],
                       prediccion=test_pd['prediction_ann'])

Resultados

In [20]:
# Resultados del modelo
dict_metrics = {'epocas':[len(history.epoch)],
                'activation':str(activation),
                'n_neurons':str(n_neuronas),
                'capas':[len(n_neuronas)],
                'training_mse':[training_metrics["mse"]],
                'training_rmse':[training_metrics["rmse"]],
                'training_mae':[training_metrics["mae"]],
                'trainig_mape':[training_metrics['mape']],
                'trainig_r':[training_metrics['r2']],
                'test_mse':[test_metrics["mse"]],
                'test_rmse':[test_metrics["rmse"]],
                'test_mae':[test_metrics["mae"]],
                'test_mape':[test_metrics['mape']],
                'test_r':[test_metrics['r2']]
                }
                
experimento_pd = pd.DataFrame.from_dict(dict_metrics)

,epocas,activation,n_neurons,capas,training_mse,training_rmse,training_mae,trainig_mape,trainig_r,test_mse,test_rmse,test_mae,test_mape,test_r
0,10,"['sigmoid', 'sigmoid']","[2, 1]",2,1.15405,1.33183,0.678521,75.169038,0.018543,1.134452,1.286982,0.672724,77.281959,0.017091


In [21]:
model_confi = {
            "n_neurons":n_neuronas,
            "activation":activation,
            "metrics":dict_metrics
            }

In [22]:
pd_trainig_test = pd.concat([trainind_pd,test_pd])

for_join = pd_historical\
                .groupby(['latitud',	'longitud', 'id_point'],as_index=False)\
                .count()[['latitud',	'longitud', 'id_point']]


pd_final = pd.merge(pd_trainig_test,for_join,on=['latitud',	'longitud'],how = 'left').sort_values(['periodo','id_point'])

pd_summary = pd.merge(pd_historical,
                        pd_final[['id_point','periodo','prediction_ann']],
                        on = ['id_point','periodo'],
                        how = 'left')

In [23]:
DIR = f'./{park}/experiments/ann'

In [24]:
import pickle

if os.listdir(DIR) == []:

    # Modelo
    model.save(f'{DIR}/model.h5')

    # Pesos
    model.save_weights(f'{DIR}/weights.h5')

    # History
    with open(f'{DIR}/history.pkl', 'wb') as file_pi:
        pickle.dump(history.history, file_pi)

    # confi
    with open(f'{DIR}/model_confi.pkl', 'wb') as file_pi:
        pickle.dump(model_confi, file_pi)
    
    # guardando resultados
    pd_summary.to_pickle(f'{DIR}/predicciones.pkl')

else:
    files = [x for x in os.listdir(f'{DIR}') if x.find('summary')!=-1 ]
    total_summary = pd.concat([pd.read_csv(f'{DIR}/{file}') for file in files])
    print( f"Actual: {test_metrics['r2']}; Best Model: {total_summary.test_r.max()}" )

    if test_metrics['r2'] > total_summary.test_r.max(): 

        # Modelo
        model.save(f'{DIR}/model.h5')

        # Pesos
        model.save_weights(f'{DIR}/weights.h5')

        # History
        with open(f'{DIR}/history.pkl', 'wb') as file_pi:
            pickle.dump(history.history, file_pi)
        
        # confi
        with open(f'{DIR}/model_confi.pkl', 'wb') as file_pi:
            pickle.dump(model_confi, file_pi)
        
        # guardando resultados
        pd_summary.to_pickle(f'{DIR}/predicciones.pkl')

Actual: 0.017090705698548758; Best Model: 0.0415621195651437


In [25]:
experi = f'{DIR}/{len(history.epoch)}_{sum(n_neuronas)}_{activation[0]}'
experimento_pd.to_csv(f'{experi}_summary.csv',index=False)